In [57]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import country_converter as coco
import warnings
warnings.filterwarnings('ignore')

In [58]:
original_df = pd.read_csv("data/stackoverflow_users_all_notext.csv", nrows=10000)

In [59]:
"""Show the data."""
display(original_df[130:140])

,id,age,location,creation_date,reputation,up_votes,down_votes
130,4845736,NaN,NaN,2015-04-29 09:41:03.96 UTC,1,0,0
131,4890609,NaN,"Belo Horizonte - State of Minas Gerais, Brazil",2015-05-12 09:03:06.03 UTC,1,0,0
132,4965490,NaN,"Talca, Chile",2015-06-02 13:36:06.94 UTC,1,0,0
133,5006338,NaN,NaN,2015-06-13 14:17:20.823 UTC,1,0,0
134,5073970,NaN,NaN,2015-07-02 13:15:12.983 UTC,1,0,0
135,5150946,NaN,"Melbourne, Australia",2015-07-24 05:58:04.17 UTC,1,0,0
136,5185772,NaN,NaN,2015-08-03 13:10:44.243 UTC,1,0,0
137,5231705,NaN,jakarta,2015-08-16 06:01:09.437 UTC,1,0,0
138,5248405,NaN,NaN,2015-08-20 16:12:00.633 UTC,1,0,0
139,5281835,NaN,NaN,2015-08-30 12:45:03.383 UTC,1,0,0


The location field seems to have the country name as the last part quite consistently, separated 
by a comma and space. Some of the countries are, however, named using some non-English form. 
Let's download a dataset of country names in different languages and use them to match locations to countries.

In [60]:
clean_df = original_df[original_df.location.notnull()]
notinteresting_df = original_df[original_df.location.isnull()]

In [61]:
"""Extract the country field out of the location field"""
clean_df["country"] = clean_df["location"].apply(lambda x: x.split(", ")[-1].lower() if isinstance(x,str) else np.nan)

In [62]:
clean_df.head(5)

,id,age,location,creation_date,reputation,up_votes,down_votes,country
1,7019478,NaN,Singapore,2016-10-14 13:36:30.96 UTC,1,0,0,singapore
2,7074992,NaN,Egypt,2016-10-26 12:11:08.41 UTC,1,0,0,egypt
3,7093392,NaN,"New York, NY, United States",2016-10-31 02:19:04.64 UTC,1,0,0,united states
6,7395536,NaN,"Cluj-Napoca, Cluj County, Romania",2017-01-09 16:36:43.747 UTC,1,0,0,romania
8,7497308,NaN,"Bodø, Norge",2017-01-31 21:13:26.053 UTC,1,0,0,norge


In [63]:
"""Load the dataset of country names in different languages got from ip2location.com."""
country_names = pd.read_csv("data/IP2LOCATION-COUNTRY-MULTILINGUAL/IP2LOCATION-COUNTRY-MULTILINGUAL.CSV")

In [64]:
display(country_names.head())

,LANG,LANG_NAME,COUNTRY_ALPHA2_CODE,COUNTRY_ALPHA3_CODE,COUNTRY_NUMERIC_CODE,COUNTRY_NAME
0,AF,AFRIKAANS,AD,AND,20,Andorra
1,AF,AFRIKAANS,AE,ARE,784,Verenigde Arabiese Emirate
2,AF,AFRIKAANS,AF,AFG,4,Afganistan
3,AF,AFRIKAANS,AG,ATG,28,Antigua en Barbuda
4,AF,AFRIKAANS,AI,AIA,660,Anguilla


In [65]:
"""Pick only the relevant part of the data"""
country_name_to_code = country_names[['COUNTRY_NAME','COUNTRY_ALPHA3_CODE']]
country_name_to_code.COUNTRY_NAME = country_name_to_code.COUNTRY_NAME.str.lower()
"""Drop duplicates since many (or probably all) countries have the same name in several languages"""
country_name_to_code = country_name_to_code[~country_name_to_code.COUNTRY_NAME.duplicated()]

In [66]:
clean_df = clean_df.merge(country_name_to_code, left_on="country", right_on="COUNTRY_NAME", how="left")
clean_df.drop(["COUNTRY_NAME"], axis=1, inplace=True)

In [67]:
clean_df.head()

,id,age,location,creation_date,reputation,up_votes,down_votes,country,COUNTRY_ALPHA3_CODE
0,7019478,NaN,Singapore,2016-10-14 13:36:30.96 UTC,1,0,0,singapore,SGP
1,7074992,NaN,Egypt,2016-10-26 12:11:08.41 UTC,1,0,0,egypt,EGY
2,7093392,NaN,"New York, NY, United States",2016-10-31 02:19:04.64 UTC,1,0,0,united states,USA
3,7395536,NaN,"Cluj-Napoca, Cluj County, Romania",2017-01-09 16:36:43.747 UTC,1,0,0,romania,ROU
4,7497308,NaN,"Bodø, Norge",2017-01-31 21:13:26.053 UTC,1,0,0,norge,NOR


In [68]:
print("Proportion of matches: ", sum(clean_df.COUNTRY_ALPHA3_CODE.notnull())/sum(clean_df.location.notnull()))

Proportion of matches:  0.8127413127413128


The method manages to add a country tag to 86.9% of the users who have informed a country in the first place. Let's then look at the users that weren't matched.

In [69]:
clean_df[clean_df.location.notnull() & clean_df.COUNTRY_ALPHA3_CODE.isnull()].head()

,id,age,location,creation_date,reputation,up_votes,down_votes,country,COUNTRY_ALPHA3_CODE
6,7579333,NaN,"Beijing, 北京市中国",2017-02-17 07:27:20.823 UTC,1,0,0,北京市中国,NaN
13,8448975,NaN,Bangalore,2017-08-11 05:32:54.82 UTC,1,0,0,bangalore,NaN
15,8623520,NaN,deverloper,2017-09-17 19:52:14.057 UTC,1,0,0,deverloper,NaN
20,9569837,NaN,UK,2018-03-29 12:56:51.147 UTC,1,0,0,uk,NaN
22,9867362,NaN,"Hangzhou, 浙江省 China",2018-05-30 01:06:04.59 UTC,1,0,0,浙江省 china,NaN


Seems like a part of the problem is people giving out only the city or some funny stuff. Let's try addressing this by loading a dataset of city names and trying to match them with the remaining, unmatched data.

In [70]:
world_cities = pd.read_csv("data/worldcities.csv")
display(world_cities.head())

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Malishevë,Malisheve,42.4822,20.7458,Kosovo,XK,XKS,Malishevë,admin,NaN,1901597212
1,Prizren,Prizren,42.2139,20.7397,Kosovo,XK,XKS,Prizren,admin,NaN,1901360309
2,Zubin Potok,Zubin Potok,42.9144,20.6897,Kosovo,XK,XKS,Zubin Potok,admin,NaN,1901608808
3,Kamenicë,Kamenice,42.5781,21.5803,Kosovo,XK,XKS,Kamenicë,admin,NaN,1901851592
4,Viti,Viti,42.3214,21.3583,Kosovo,XK,XKS,Viti,admin,NaN,1901328795


In [71]:
display(world_cities[world_cities.city=="Moscow"])

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
2220,Moscow,Moscow,55.7522,37.6155,Russia,RU,RUS,Moskva,primary,10452000.0,1643318494
11641,Moscow,Moscow,46.7307,-116.9986,United States,US,USA,Idaho,NaN,25579.0,1840019868


There are cities with the same names in many countries. Let's assume that if no country was said, it's probably the largest city.
One could in principle also use the administrative area name here but it doesn't seem as likely that people would put just the name of the administrative area but not the country or city name so let's try the city first.

In [72]:
world_cities = world_cities.sort_values(by="population").drop_duplicates(subset="city",keep="last").sort_index()

In [73]:
"""Again, take only interesting parts"""
city_to_country_code = world_cities[['city','iso3']]
city_to_country_code.city = city_to_country_code.city.str.lower()

In [74]:
clean_df["city_candidate"] = clean_df["location"].apply(lambda x: x.split(", ")[0].lower() if isinstance(x,str) else np.nan)

In [75]:
clean_df = clean_df.merge(city_to_country_code, left_on="city_candidate", right_on="city", how="left")
clean_df["COUNTRY_ALPHA3_CODE"][clean_df["COUNTRY_ALPHA3_CODE"].isnull()] = clean_df["iso3"][clean_df["COUNTRY_ALPHA3_CODE"].isnull()]

In [76]:
print("Proportion of matched: ", sum(clean_df.COUNTRY_ALPHA3_CODE.notnull())/sum(clean_df.location.notnull()))

Proportion of matched:  0.9146718146718147


Seems like using the city name brings us up to 92.3% of matches.

In [77]:
"""Clean out some useless fields"""
clean_df.drop(["city_candidate", "city", "iso3","country"],axis=1, inplace=True)

In [78]:
"""See the remaining users that are still unmatched. Also prepare a lowercase location column for further analysis."""
clean_df["location_lc"] = clean_df.location.str.lower()
remaining = clean_df[clean_df.location.notnull()][clean_df.COUNTRY_ALPHA3_CODE.isnull()]
display(remaining[100:110])

,id,age,location,creation_date,reputation,up_votes,down_votes,COUNTRY_ALPHA3_CODE,location_lc
1267,3562541,NaN,Minnesota,2014-04-23 00:39:46.953 UTC,3,0,0,NaN,minnesota
1272,5749992,NaN,Somewhere Over the Rainbow,2016-01-05 23:47:53.463 UTC,3,0,0,NaN,somewhere over the rainbow
1280,8821565,NaN,"Taipei 101, Section 5, Xinyi Road, Xinyi Distr...",2017-10-23 20:07:19.247 UTC,3,0,0,NaN,"taipei 101, section 5, xinyi road, xinyi distr..."
1282,4366341,NaN,Reek,2014-12-16 12:33:32.24 UTC,3,0,0,NaN,reek
1286,4593809,NaN,"Hull, UK",2015-02-22 13:56:59.387 UTC,3,0,0,NaN,"hull, uk"
1287,7570131,NaN,"Walchand College of Engineering ,sangli-416415...",2017-02-15 16:22:29.603 UTC,3,0,0,NaN,"walchand college of engineering ,sangli-416415..."
1295,1792154,NaN,Idaho,2012-11-01 17:49:20.17 UTC,3,0,0,NaN,idaho
1296,2299708,NaN,UT,2013-04-19 14:55:46.24 UTC,3,0,0,NaN,ut
1303,2388579,NaN,Studio City,2013-05-16 06:00:49.067 UTC,3,0,0,NaN,studio city
1311,3330609,NaN,Online,2014-02-20 00:48:26.767 UTC,3,0,0,NaN,online


Seems like the remaining problem is in part that the format isn't necessarily
always city, admin area, country but sometimes some other ordering. Also, for instance UK is a common abbreviation for the United Kingdom but it wasn't in the country dataset. A regex search for the rest seems like a good idea.

In [79]:
country_name_to_code_2 = country_name_to_code.append([{"COUNTRY_NAME" : "uk", "COUNTRY_ALPHA3_CODE" : "GBR"},
                                                     {"COUNTRY_NAME" : "usa", "COUNTRY_ALPHA3_CODE" : "USA"},
                                                     {"COUNTRY_NAME" : "south korea", "COUNTRY_ALPHA3_CODE" : "KOR"}] ,ignore_index=True)

In [ ]:
i = 0
for idx, row in country_name_to_code_2.iterrows():
    country_name,iso3 = row[0],row[1]
    matching = remaining.location_lc.str.contains("\\b"+country_name+"\\b")
    remaining.COUNTRY_ALPHA3_CODE[matching] = iso3
    if i % 1000 == 0:
        print(idx/len(country_name_to_code)*100, "% calculated")
    i += 1

0.0 % calculated
7.471049682480388 % calculated
14.942099364960775 % calculated


In [26]:
clean_df = clean_df.merge(remaining.drop(["id","location","creation_date","reputation","up_votes","down_votes","location_lc","age"],axis=1), left_index=True, right_index=True, how="left")
clean_df.COUNTRY_ALPHA3_CODE_x[clean_df.COUNTRY_ALPHA3_CODE_x.isnull()] = clean_df.COUNTRY_ALPHA3_CODE_y[clean_df.COUNTRY_ALPHA3_CODE_x.isnull()]

In [27]:
clean_df.drop(["COUNTRY_ALPHA3_CODE_y"], axis=1, inplace=True)
clean_df.rename(columns={"COUNTRY_ALPHA3_CODE_x" : "country_iso3"}, inplace=True)

In [28]:
print("Proportion of matched: ", sum(clean_df.country_iso3.notnull())/sum(clean_df.location.notnull()))

Proportion of matched:  0.9474566236164214


Using the regex method brings the results already to 94.75 percent. Let's look at the remaining issues.

In [29]:
clean_df[clean_df.location.notnull() & clean_df.country_iso3.isnull()][160:180]

,id,age,location,creation_date,reputation,up_votes,down_votes,country_iso3,location_lc
8380,9333556,NaN,Czechia,2018-02-08 14:04:47.567 UTC,30,0,0,NaN,czechia
8409,9719937,NaN,Worldwide,2018-04-30 06:38:27.79 UTC,30,47,0,NaN,worldwide
8538,5991722,NaN,"Hranice na Moravě, Hranice, Česko",2016-02-27 22:37:37.31 UTC,461,27,17,NaN,"hranice na moravě, hranice, česko"
8622,6292000,NaN,Somewhere,2016-05-04 17:01:26.607 UTC,308,693,0,NaN,somewhere
8732,6783798,NaN,Schweinfurt,2016-09-01 14:29:22.53 UTC,1061,129,26,NaN,schweinfurt
8800,7165275,NaN,Vermont,2016-11-16 03:00:26.087 UTC,1943,60,12,NaN,vermont
8901,7771928,NaN,Bangaore,2017-03-27 04:26:53.803 UTC,37,0,0,NaN,bangaore
8924,7878102,NaN,"Nagrota Bypass Road, Jammu",2017-04-17 10:18:59.57 UTC,61,8,0,NaN,"nagrota bypass road, jammu"
8931,7898657,NaN,中国河北省Shijiazhuang Shi,2017-04-21 00:05:31.497 UTC,431,4,3,NaN,中国河北省shijiazhuang shi
8947,7974340,NaN,U.S.,2017-05-06 20:41:23.647 UTC,126,12,0,NaN,u.s.


Seems like the remaining issues are mostly people giving out only the city or administrative area name in some format that wasn't considered before. Let's try to fix this with a similar regex matching for both.

In [30]:
world_cities_2 = pd.read_csv("data/worldcities.csv")
world_cities_2 = world_cities.sort_values(by="population").drop_duplicates(subset="admin_name",keep="last").sort_index()
admin_to_country_code = world_cities_2[['admin_name','iso3']]
admin_to_country_code.admin_name = admin_to_country_code.admin_name.str.lower()

In [32]:
"""Weird thing: NaN value in one of the admin names"""
remaining = clean_df[clean_df.location_lc.notnull()][clean_df.country_iso3.isnull()]
i = 0
for idx, row in admin_to_country_code.iterrows():
    admin_name,iso3 = row[0],row[1]
    matching = remaining.location_lc.str.contains("\\b"+str(admin_name)+"\\b")
    remaining.country_iso3[matching] = iso3
    if i % 1000 == 0:
        print(i/len(admin_to_country_code)*100, "% calculated")
    i += 1

0.0 % calculated
25.73340195573855 % calculated
51.4668039114771 % calculated
77.20020586721564 % calculated


In [33]:
remaining.rename(columns={"country_iso3" : "rem_iso3"}, inplace=True)

In [34]:
clean_df = clean_df.merge(remaining.drop(["id","location","creation_date","reputation","up_votes","down_votes","location_lc","age"],axis=1), left_index=True, right_index=True, how="left")
clean_df.country_iso3[clean_df.country_iso3.isnull()] = clean_df.rem_iso3[clean_df.country_iso3.isnull()]
clean_df.drop(["rem_iso3"], axis=1, inplace=True)

In [35]:
remaining = clean_df[clean_df.location_lc.notnull() & clean_df.country_iso3.isnull()]
i = 0
for idx, row in city_to_country_code.iterrows():
    city_name,iso3 = row[0],row[1]
    matching = remaining.location_lc.str.contains("\\b"+city_name+"\\b")
    remaining.country_iso3[matching] = iso3
    if i % 1000 == 0:
        print(i/len(city_to_country_code)*100, "% calculated")
    i += 1

0.0 % calculated
8.634832915983075 % calculated
17.26966583196615 % calculated
25.90449874794923 % calculated
34.5393316639323 % calculated
43.17416457991538 % calculated
51.80899749589846 % calculated
60.44383041188153 % calculated
69.0786633278646 % calculated
77.71349624384769 % calculated
86.34832915983075 % calculated
94.98316207581384 % calculated


In [36]:
remaining.rename(columns={"country_iso3" : "rem_iso3"}, inplace=True)

In [37]:
clean_df = clean_df.merge(remaining.drop(["id","location","creation_date","reputation","up_votes","down_votes","location_lc","age"],axis=1), left_index=True, right_index=True, how="left")
clean_df.country_iso3[clean_df.country_iso3.isnull()] = clean_df.rem_iso3[clean_df.country_iso3.isnull()]
clean_df.drop(["rem_iso3"], axis=1, inplace=True)

In [38]:
display(admin_to_country_code[[not isinstance(x,str) for x in admin_to_country_code.admin_name]])

,admin_name,iso3
7818,NaN,JEY


Funny fact: before we needed to have str(admin_name) in the code because there's a place called Nan in Thailand. 

In [39]:
sum(clean_df.country_iso3.notnull())/sum(clean_df.location.notnull())

0.956600201369446

With the final method we get 95.66% of country matches to locations.

In [40]:
clean_df.drop(["location","location_lc"],axis=1,inplace=True)

In [41]:
clean_df.to_csv("data/stackoverflow_users_2016-2019_notext_clean.csv")

In [42]:
original_df[clean_df.country_iso3.isnull() & original_df.location.notnull()]

,id,age,location,creation_date,reputation,up_votes,down_votes,country
32,5876676,NaN,中国广东省Guangzhou Shi,2016-02-03 07:47:09.317 UTC,92,86,0,中国广东省guangzhou shi
49,5924453,NaN,Hobbiton,2016-02-14 06:50:46.743 UTC,610,40,5,hobbiton
244,7008814,NaN,mars,2016-10-12 15:28:03.683 UTC,15,68,0,mars
425,7878292,NaN,Arandjelovac,2017-04-17 11:10:19.7 UTC,18,3,0,arandjelovac
435,7945249,NaN,127.0.0.1,2017-04-30 22:50:32.76 UTC,154,7,0,127.0.0.1
...,...,...,...,...,...,...,...,...
6035679,6142801,NaN,"New Jersey, United States",2016-04-01 01:57:26.287 UTC,31,0,0,united states
6035680,9703547,NaN,"Yokohama, Kanagawa Prefecture, Japan",2018-04-26 10:47:03.497 UTC,31,2,0,japan
6035682,11258913,NaN,"Mississippi, USA",2019-03-26 07:21:08.673 UTC,31,24,0,usa
6035685,9862136,NaN,"Everett, 워싱턴 미국",2018-05-29 04:55:22.243 UTC,31,10,0,워싱턴 미국


In [43]:
original_df[original_df.location.notnull() & clean_df.country_iso3.isnull()][100:130]

,id,age,location,creation_date,reputation,up_votes,down_votes,country
6163,8644091,NaN,Babu Nagar,2017-09-20 19:28:47.367 UTC,429,166,19,babu nagar
6174,8813176,NaN,Mariana Trench,2017-10-22 03:32:06.66 UTC,93,10,0,mariana trench
6180,8867531,NaN,Czechia,2017-11-01 10:38:22.473 UTC,29,18,0,czechia
6203,9015220,NaN,Korean,2017-11-27 14:22:04.717 UTC,55,9,0,korean
6211,9061350,NaN,Malu abrupt,2017-12-06 11:10:36.39 UTC,36,0,0,malu abrupt
6262,9537794,NaN,Everywhere,2018-03-23 00:48:57.613 UTC,55,2,0,everywhere
6309,9975719,NaN,DC,2018-06-21 23:43:00.367 UTC,30,2,0,dc
6327,10211923,NaN,HumanLand,2018-08-11 10:33:01.33 UTC,632,74,10,humanland
6356,5789789,NaN,GetPosition,2016-01-14 12:56:40.593 UTC,23,0,0,getposition
6446,6176383,NaN,Milky-Way,2016-04-08 09:17:07.11 UTC,96,5,0,milky-way


Looks that most of the unmatched entries are just people trying to be funny. "Earth" and "Moon" are quite popular locations, for instance.

In [44]:
world_cities[world_cities.city.str.lower().str.contains("\\bada\\b")]

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
2010,Ada,Ada,45.8027,20.1285,Serbia,RS,SRB,Ada,admin,NaN,1688687750


There's also a city called Ada in Serbia. :)